In [6]:
import os
import json

with open(".env.json", "r") as f_in:
    env_json: dict = json.load(f_in)
    for key, value in env_json.items():
        os.environ[key] = value

In [7]:
import pandas as pd
df_dataset_val = pd.DataFrame([
    ["hola", "hola como estas"]
], columns=['input', 'expected_output'])
df_dataset_val.head()

,input,expected_output
0,hola,hola como estas


In [8]:
from framework_genai.GenAIMemory import GenAIMemory, EnumMemoryType
from framework_genai.GenAILLM import GenAILLM, EnumGenAIPlatforms, EnumGenAIModelsIdsOpenAI
from framework_genai.LoggingAndTelemetry import EnumLogs
import os

ai_prefix = "Human"
human_prefix = "Assistant"

gen_ai_llm = GenAILLM(
    platform = EnumGenAIPlatforms.PLATFORM_OPENAI,
    model_id = EnumGenAIModelsIdsOpenAI.MODEL_CHAT_GPT_3_5_TURBO,
    parameters_inference = {
        'max_tokens':512, 
        "temperature": 0,
        "top_p": 0.1
    },
    ai_prefix = ai_prefix,
    human_prefix = human_prefix,
    component_memory=GenAIMemory(EnumMemoryType.MEMORY_CHAT_BUFFER, ai_prefix, human_prefix),
    platform_configuration = {
        "OPENAI_API_KEY": os.environ["OPENAI_API_KEY"]
    },
    verify_ssl=True,
    verbose_level=EnumLogs.LOG_LEVEL_DEBUG
)
print(gen_ai_llm.llm_model.verbose)
# print(gen_ai_llm.invoke("Hola", return_completion_only=True, save_in_memory=False))

2024-02-08 05:44:57.474344-05:00: Modelo EnumGenAIModelsIdsOpenAI.MODEL_CHAT_GPT_3_5_TURBO construido
True


In [9]:
def pseudo_invoke(input):
    return gen_ai_llm.invoke("Hola", return_completion_only=True, save_in_memory=False)
df_dataset_val["actual_output"] = df_dataset_val["input"].apply(pseudo_invoke)
df_dataset_val.head()

2024-02-08 05:44:57.500333-05:00: Invocando modelo EnumGenAIModelsIdsOpenAI.MODEL_CHAT_GPT_3_5_TURBO con Hola


,input,expected_output,actual_output
0,hola,hola como estas,¡Hola! ¿En qué puedo ayudarte hoy?


In [11]:
list(df_dataset_val["actual_output"].values)

['¡Hola! ¿En qué puedo ayudarte hoy?']

In [12]:
from bert_score import score

P, R, F1 = score(list(df_dataset_val["actual_output"].values), list(df_dataset_val["expected_output"].values), lang="es", verbose=True)
print(P, R, F1)

calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:16<00:00, 16.26s/it]


computing greedy matching.


100%|██████████| 1/1 [00:09<00:00,  9.04s/it]


done in 25.64 seconds, 0.04 sentences/sec
tensor([0.6277]) tensor([0.6612]) tensor([0.6440])
